In [ ]:
import numpy as np
from pathlib import Path
import re
import matplotlib.pyplot as plt

In [ ]:
train_log_dir = Path("exp/ywt_stock/graph_nodes_v_mode_None/my_exp_none-tra-_gnn_tra_mlp_202304180759/")
train_log_f = train_log_dir / "log.txt"
with open(train_log_f) as f:
    log_text = f.read()

best_epoch = re.findall("(?<=Best Epoch: ).+", log_text)[0]
best_model_performance = re.sub(",\sepoch time:\s\d+\.\d+s", "", re.findall(f"(?<=Epoch: {best_epoch},).+", log_text)[0])
best_model_performance = re.sub(",\stotal time:\s\d+\.\d+s", "", best_model_performance)
model_name = train_log_dir.stem
main_title = model_name + "\n" + re.sub(", roc:.+", "", best_model_performance).rstrip("|| ").replace("||", "\n")
print(best_epoch)
print(main_title)
record_fields = ["acc_train", "acc_val", "delta_train", "delta_val", "mse_train", "mse_val", "emb_train", "emb_val", "roc", "dist", "tri"]
fig_title = ["Train edge accuracy", "Val edge accuracy",
             "Train correlation matrix loss", "Val correlation matrix loss",
             "Train nodes values matrix loss", "Val nodes values matrix loss",
             "Train embedding disparity between spatial & temporal", "Val embedding disparity between spatial & temporal",
             "AUC", "DIST",
             "TRI"]
data_info = list(map(lambda r, t: (np.array([float(s) for s in re.findall(f"(?<={r}: ).*?(?=,)", log_text)]), t), record_fields, fig_title))
append_data_info = [(data_info[2][0] + data_info[4][0], "Train total loss"), (data_info[3][0] + data_info[5][0], "Validation total loss")]
data_info += append_data_info
data_info[-2], data_info[-1], data_info[8], data_info[9] = data_info[8], data_info[9], data_info[-2], data_info[-1]
fig, axs = plt.subplot_mosaic("""
                              ab
                              cd
                              ef
                              gh
                              ij
                              kl
                              mn
                              """,
                              layout='constrained', figsize=(20, 25))
fig.suptitle(main_title, fontsize=28)

for ax, (data, title) in zip(axs.values(), data_info):
    ax.plot(data)
    ax.set_title(title, fontsize=20)
plt.show()
plt.close()